

## **Chapter 9: Authentication and Authorization**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Distinguish between Authentication (identity) and Authorization (permissions) in GraphQL
- Implement JWT-based authentication in a GraphQL server
- Inject authenticated user data into the GraphQL Context
- Apply authorization logic using middleware patterns
- Protect specific fields and operations using resolver-level checks
- Create custom authorization directives for declarative permission management
- Implement Role-Based Access Control (RBAC) systems
- Secure your GraphQL API against common authentication vulnerabilities

---

## **Prerequisites**

- Completed Chapter 7: Building a GraphQL Server
- Completed Chapter 8: Error Handling and Validation
- Understanding of HTTP headers and token-based authentication concepts
- Basic knowledge of JWT (JSON Web Tokens) structure

---

## **9.1 Understanding Authentication vs. Authorization**

Before writing code, it is critical to distinguish between these two security concepts. They are often confused but serve different purposes.

### **Authentication: "Who are you?"**

Authentication is the process of verifying the identity of a user or service. In GraphQL, this typically happens **before** the query execution begins.

**Common Methods:**
*   **API Keys:** Simple tokens for service-to-service communication.
*   **JWT (JSON Web Tokens):** Stateless tokens containing user claims (ID, role, expiration).
*   **Session Cookies:** Server-side storage of user login state.

**The GraphQL Context:**
Authentication is the bridge between the HTTP layer and your GraphQL layer. You extract credentials from the HTTP request (usually the `Authorization` header) and place the verified user into the `context` object.

### **Authorization: "What can you do?"**

Authorization determines if the authenticated user has permission to perform a specific action or access specific data. In GraphQL, this happens **during** execution.

**Levels of Authorization:**
1.  **Operation-Level:** Can the user run this Query/Mutation? (e.g., "Only admins can delete users").
2.  **Field-Level:** Can the user see this field? (e.g., "Only the owner can see their email address").
3.  **Object-Level:** Can the user access this specific object? (e.g., "Only the author can edit this post").

---

## **9.2 Implementing Authentication (JWT Strategy)**

We will implement a robust authentication flow using **JWT (JSON Web Tokens)**. This is the industry standard for modern, stateless GraphQL APIs.

### **Setting up the Authentication Flow**

The flow works as follows:
1.  **Login Mutation:** Client sends credentials (email/password).
2.  **Verification:** Server verifies credentials against the database.
3.  **Token Generation:** If valid, server generates a JWT and returns it to the client.
4.  **Authenticated Requests:** Client sends the JWT in the `Authorization` header of subsequent requests.
5.  **Context Injection:** The server middleware validates the JWT and injects the user into the `context`.

### **Step 1: The Login Mutation**

First, we need a way for users to get a token.

**Schema:**
```graphql
type AuthPayload {
  token: String!
  user: User!
}

type Mutation {
  login(email: String!, password: String!): AuthPayload!
}
```

**Resolver Implementation:**
We use `bcrypt` to compare the plain text password with the hashed password stored in the database.

```javascript
const bcrypt = require('bcryptjs');
const jwt = require('jsonwebtoken');

// Ideally stored in environment variables
const JWT_SECRET = 'super_secret_key';

const resolvers = {
  Mutation: {
    login: async (_, { email, password }, { dataSources }) => {
      // 1. Find user by email
      const user = await dataSources.userAPI.findUserByEmail(email);
      
      if (!user) {
        throw new AuthenticationError('No user found with this email');
      }

      // 2. Compare password with stored hash
      const isValid = await bcrypt.compare(password, user.passwordHash);
      
      if (!isValid) {
        throw new AuthenticationError('Invalid password');
      }

      // 3. Generate JWT
      const token = jwt.sign(
        { userId: user.id, role: user.role }, // Payload
        JWT_SECRET,                           // Secret
        { expiresIn: '1h' }                   // Options
      );

      // 4. Return token and user
      return {
        token,
        user,
      };
    },
  },
};
```

**Explanation:**
*   **`bcrypt.compare()`**: Safely compares a plain text string with a hash. It handles the salt automatically and prevents timing attacks.
*   **`jwt.sign()`**: Creates the token.
    *   **Payload:** We store minimal data (userId, role). Never store sensitive data like passwords in the token payload.
    *   **Secret:** A secret key used to sign the token. Must be complex and stored securely (environment variables).
    *   **Expiration:** Sets how long the token is valid (`expiresIn: '1h'`). Shorter is safer.
*   **`AuthenticationError`**: A standard Apollo error class that signifies the client is not authenticated.

### **Step 2: Extracting Tokens in the Context**

Now that the client has a token, they will send it in the HTTP header. We must extract it on the server.

**Server Setup (Apollo Server):**

```javascript
const { ApolloServer, AuthenticationError } = require('apollo-server');
const jwt = require('jsonwebtoken');

const server = new ApolloServer({
  typeDefs,
  resolvers,
  
  // The context function runs for EVERY request
  context: async ({ req }) => {
    // 1. Get the token from the Authorization header
    // Header format: "Bearer <token>"
    const authHeader = req.headers.authorization || '';
    const token = authHeader.replace('Bearer ', '');

    // 2. If no token, return empty context (user is null)
    if (!token) {
      return { user: null };
    }

    // 3. Verify the token
    try {
      const decoded = jwt.verify(token, JWT_SECRET);
      
      // decoded will look like: { userId: '123', role: 'ADMIN', iat: ..., exp: ... }
      
      // 4. Optionally, fetch the full user from DB to ensure they still exist
      const user = await getUserFromDatabase(decoded.userId);

      // 5. Return the user in the context
      return { user };
      
    } catch (err) {
      // Token is invalid or expired
      throw new AuthenticationError('Invalid or expired token');
    }
  }
});

server.listen().then(({ url }) => {
  console.log(`Server ready at ${url}`);
});
```

**Explanation:**
*   **`req.headers.authorization`**: Accesses the HTTP headers sent by the client.
*   **Token format**: The standard convention is to prefix the token with `Bearer `. We strip this prefix before verification.
*   **`jwt.verify()`**: Decodes and validates the token. If the signature is invalid or the token is expired, it throws an error.
*   **Context return**: We return an object containing the `user`. This object is now available as the third argument in **every** resolver.

---

## **9.3 Authorization Strategies**

Once the `user` is in the context, we need to restrict access. There are three main strategies, ranging from simple to advanced.

### **9.3.1 The Middleware Approach (Simple & Effective)**

The most straightforward way to protect operations is to check the context at the beginning of the resolver.

**Scenario:** Only authenticated users can view their own profile.

```javascript
const resolvers = {
  Query: {
    // The 'me' query returns the current user
    me: (parent, args, context) => {
      // Check if user exists in context
      if (!context.user) {
        throw new AuthenticationError('You must be logged in');
      }
      return context.user;
    },
    
    // Only admins can see all users
    users: (parent, args, context) => {
      if (!context.user || context.user.role !== 'ADMIN') {
        throw new ForbiddenError('You are not authorized to view this data');
      }
      return dataSources.userAPI.getAllUsers();
    }
  }
};
```

**Pros:**
*   Easy to implement and understand.
*   Co-located with the logic.

**Cons:**
*   Repetitive (checking `if (!context.user)` in every resolver).
*   Easy to forget to add the check to a new resolver.

### **9.3.2 Field-Level Authorization (Data Logic)**

Sometimes, authorization isn't about *if* you can query a user, but *what fields* you can see. For example, a user's email might be public for admins but private for others.

**Resolver Approach:**

```javascript
const resolvers = {
  User: {
    // This resolver only runs when the 'email' field is requested
    email: (user, args, context) => {
      // Logic: User can see own email, Admins can see all emails
      const isSelf = context.user && context.user.id === user.id;
      const isAdmin = context.user && context.user.role === 'ADMIN';
      
      if (!isSelf && !isAdmin) {
        // Return null or throw an error
        return null; 
        // OR: throw new ForbiddenError('Cannot access this field');
      }
      
      return user.email;
    }
  }
};
```

**Explanation:**
*   **Granular Control:** This allows you to return a partial object. The client gets the User object, but the `email` field is `null` if they aren't authorized.
*   **Resolver Chaining:** This specific resolver for the `email` field only runs if `email` is requested in the query, saving performance.

### **9.3.3 The Directive Approach (Declarative & Reusable)**

For larger APIs, checking permissions manually in every resolver is unmaintainable. **Directives** allow you to declare permissions directly in your schema.

**Goal:** We want to write this schema:
```graphql
type Query {
  users: [User!]! @auth(requires: ADMIN)
  me: User @auth
}

type User {
  id: ID!
  email: String @auth(requires: ADMIN) # Only admins see email
}
```

**Implementation (Apollo Server):**

Directives require two parts: the Schema definition and the Resolver logic class.

**1. Define the Directive Schema:**

```graphql
enum Role {
  ADMIN
  USER
}

directive @auth(requires: Role = USER) on FIELD_DEFINITION | OBJECT
```

**2. Implement the Directive Logic Class:**

```javascript
const { SchemaDirectiveVisitor, ForbiddenError } = require('apollo-server');
const { defaultFieldResolver } = require('graphql');

class AuthDirective extends SchemaDirectiveVisitor {
  // This method is called when the directive is encountered on a field
  visitFieldDefinition(field) {
    const requiredRole = this.args.requires;
    
    // Get the original resolver (or default resolver)
    const originalResolver = field.resolve || defaultFieldResolver;

    // Replace the resolver with our custom wrapper
    field.resolve = async function (parent, args, context, info) {
      // 1. Authentication Check
      if (!context.user) {
        throw new AuthenticationError('You must be logged in');
      }

      // 2. Authorization Check
      const userRole = context.user.role; 
      if (userRole !== requiredRole && requiredRole !== 'USER') {
        // If required role is ADMIN and user is not ADMIN
        throw new ForbiddenError(`Requires ${requiredRole} role`);
      }

      // 3. If checks pass, run the original resolver
      return originalResolver(parent, args, context, info);
    };
  }
}
```

**3. Apply Directive to Server:**

```javascript
const server = new ApolloServer({
  typeDefs,
  resolvers,
  schemaDirectives: {
    auth: AuthDirective, // Register the directive
  },
  context: ... // (Context logic from previous section)
});
```

**Explanation:**
*   **`SchemaDirectiveVisitor`**: A class that allows you to modify the schema at startup.
*   **`visitFieldDefinition`**: This hook runs once for every field marked with `@auth`.
*   **Wrapper Pattern**: We "wrap" the original resolver. Our wrapper runs the security checks first. If they pass, we hand over control to the original resolver.
*   **Declarative**: Now, security is defined in the schema, making it self-documenting and centralized.

---

## **9.4 Role-Based Access Control (RBAC) Implementation**

RBAC is the standard model for managing permissions in enterprise applications. Instead of checking for specific permissions (`CAN_DELETE_USER`), we check for Roles (`ADMIN`), and Roles have permissions.

### **The RBAC Data Model**

**1. Define Permissions (Enum):**
```graphql
enum Permission {
  READ_OWN_PROFILE
  READ_ANY_PROFILE
  EDIT_OWN_PROFILE
  MANAGE_USERS
  DELETE_POSTS
}
```

**2. Assign Permissions to Roles:**
This logic can live in code or a database.

```javascript
const ROLE_PERMISSIONS = {
  GUEST: ['READ_OWN_PROFILE'],
  USER: ['READ_OWN_PROFILE', 'EDIT_OWN_PROFILE', 'READ_ANY_PROFILE'],
  ADMIN: ['READ_OWN_PROFILE', 'EDIT_OWN_PROFILE', 'READ_ANY_PROFILE', 'MANAGE_USERS', 'DELETE_POSTS'],
};

function hasPermission(userRole, requiredPermission) {
  const permissions = ROLE_PERMISSIONS[userRole] || [];
  return permissions.includes(requiredPermission);
}
```

**3. Using RBAC in Resolvers:**

```javascript
const resolvers = {
  Mutation: {
    deleteUser: async (_, { id }, context) => {
      // Check for specific permission
      if (!hasPermission(context.user.role, 'MANAGE_USERS')) {
        throw new ForbiddenError('You do not have permission to manage users');
      }
      
      return context.dataSources.userAPI.deleteUser(id);
    }
  }
};
```

**4. Advanced Directive for Permissions:**

You can extend the `@auth` directive to accept permissions instead of just roles.

```graphql
directive @hasPermission(perm: Permission) on FIELD_DEFINITION

type Mutation {
  deleteUser(id: ID!): User @hasPermission(perm: MANAGE_USERS)
}
```

---

## **9.5 Security Best Practices**

1.  **HTTPS is Mandatory:** Never send JWTs or credentials over HTTP. They will be intercepted.
2.  **Short Token Expiration:** Access tokens should expire quickly (e.g., 15 minutes - 1 hour). Use "Refresh Tokens" for long sessions.
3.  **Secure Storage:** On the client side, do not store tokens in `localStorage` (vulnerable to XSS). Use `HttpOnly` cookies if possible.
4.  **Query Depth Limiting:** Authentication prevents unauthorized access, but authenticated users can still crash your server with deeply nested queries. Use depth limiting libraries (like `graphql-depth-limit`).
5.  **Introspection in Production:** Disable the introspection query in production to prevent attackers from discovering your entire schema structure.
    ```javascript
    new ApolloServer({
      // ...
      introspection: process.env.NODE_ENV !== 'production',
    });
    ```
6.  **Hash Passwords Properly:** Always use `bcrypt` or `argon2`. Never store plain text passwords. Never invent your own hashing algorithm.

---

## **Chapter Summary**

In this chapter, we secured our GraphQL API by implementing a complete security lifecycle:

### **Key Takeaways:**

1.  **Authentication vs Authorization**: Authentication verifies identity (Who are you?), while Authorization verifies permissions (What can you do?).
2.  **JWT Implementation**: We implemented a stateless authentication flow using JSON Web Tokens, handling login, token generation, and verification.
3.  **The Context Object**: The `context` function is the secure bridge between HTTP and GraphQL. It is where authentication logic lives.
4.  **Authorization Strategies**:
    *   **Resolver Checks**: Simple and co-located, but can be repetitive.
    *   **Field-Level**: Granular control over specific data fields.
    *   **Directives**: The most scalable and declarative approach for large APIs.
5.  **RBAC**: We implemented Role-Based Access Control to manage permissions systematically.
6.  **Security Hygiene**: We covered essential practices like HTTPS, short token lifespans, and disabling introspection in production.

### **Best Practices:**

*   Never store sensitive data in JWT payloads.
*   Use `bcrypt` for password hashing.
*   Centralize authentication logic in the `context` function.
*   Prefer Directives for authorization to keep schema logic clean.
*   Always validate that the authenticated user still exists in the database during context creation.

---

### **🚀 Next Up: Chapter 10 - The Client Ecosystem**

**Summary:** Our server is secure and robust. Now it's time to build the consumer. In Chapter 10, we shift to the frontend. We will explore the GraphQL client ecosystem, focusing on Apollo Client. You will learn how to set up the client, perform queries and mutations, handle loading and error states in the UI, and manage the Apollo Cache for a lightning-fast user experience.